# Part 1

In [1]:
# Stolen from day 6 and modified for today.
import itertools

class Matrix:
    ''' A dense matrix. '''
    def __init__(self, rows, cols, initial=' '):
        self._rows = rows
        self._cols = cols
        self._cells = [[initial] * cols for _ in range(rows)]
    
    def __repr__(self):
        return 'Matrix<{}x{}>'.format(self._rows, self._cols)
    
    def __str__(self):
        render = ''
        for row in self._cells:
            render += ''.join(str(v) for v in row) + '\n'
        return render

    def __getitem__(self, key):
        row, col = key
        if row < 0 or row >= self._rows or col < 0 or col >= self._cols:
            raise KeyError(key)
        return self._cells[row][col]
    
    def __setitem__(self, key, val):
        row, col = key
        if row < 0 or row >= self._rows or col < 0 or col >= self._cols:
            raise KeyError(key)
        self._cells[row][col] = val
    
    def clone(self):
        m = Matrix(self._rows, self._cols)
        for row, col in itertools.product(range(self._rows), range(self._cols)):
            m[row,col] = self[row,col]
        return m

    def get(self, row, col, default=None):
        try:
            return self[row,col]
        except (IndexError,KeyError):
            return default
    
    @property
    def size(self):
        return self._rows, self._cols

    def values(self):
        for row, col in itertools.product(range(self._rows), range(self._cols)):
            yield self[row, col]

In [2]:
class Cart:
    char2dir = {
        '^': 0,
        '>': 1,
        'v': 2,
        '<': 3,
    }
    dir2char = {v:k for k,v in char2dir.items()}
    CART_CHARS = set(char2dir.keys())

    def __init__(self, initial):
        self.force_turn(initial)
        self._next_turn = 0
    
    def __repr__(self):
        return '<Cart direction={} next_turn={}>'.format(
            Cart.dir2char[self._direction], self._next_turn)

    def __str__(self):
        return Cart.dir2char[self._direction]

    def force_turn(self, char):
        ''' Force the cart to turn, i.e. when it's in a corner. '''
        self._direction = Cart.char2dir[char]
    
    def choose_turn(self):
        ''' Allow the cart to choose which way to turn at an intersection. '''
        if self._next_turn == 0:
            # turn left
            self._direction = (self._direction - 1) % 4
        elif self._next_turn == 1:
            # go straight
            pass
        elif self._next_turn == 2:
            # go right
            self._direction = (self._direction + 1) % 4
        else:
            raise Exception('Invalid next turn: {}'.format(self._next_turn))
        self._next_turn = (self._next_turn + 1) % 3

In [3]:
RAIL_CHARS = '|-/\\+'

def parse_map(text):
    ''' Parse rail map into a matrix and a dict of carts. '''
    lines = text.split('\n')
    rows = len(lines)
    cols = max(len(line) for line in lines)
    mat = Matrix(rows, cols)
    carts = dict()
    for row, line in enumerate(lines):
        for col, char in enumerate(line):
            if char == ' ':
                continue
            if char in RAIL_CHARS:
                mat[row,col] = char
            elif char in Cart.CART_CHARS:
                carts[row,col] = Cart(char)
                if char in '<>':
                    mat[row,col] = '-'
                else:
                    mat[row,col] = '|'
    return mat, carts

def print_map(mat, carts):
    ''' Render a map of the rails and carts. '''
    lines = [list(line) for line in str(mat).split('\n')]
    for (row, col), cart in carts.items():
        lines[row][col] = str(cart)
    print('\n'.join(''.join(line) for line in lines))

In [21]:
test_text = r'''/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/'''
test_mat, test_carts = parse_map(test_text)
print_map(test_mat, test_carts)
for (row, col), test_cart in test_carts.items():
    print(row, col, repr(test_cart))

/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   

0 2 <Cart direction=> next_turn=0>
3 9 <Cart direction=v next_turn=0>


In [50]:
TURN_CHARS = '/\\'

def move_carts(mat, carts):
    ''' Do one round of moving carts. If a collision occurs, return the coordinate
    of the collision immediately. '''
    # Make sure we process carts in the correct order.
    cart_list = [(row, col, cart) for (row,col), cart in carts.items()]
    cart_list.sort()
    for row, col, cart in cart_list:
        cell = mat[row, col]
        char = str(cart)
        # Does the cart need to change direction?
        if cell == '/':
            if char == '^':
                cart.force_turn('>')
            elif char == '>':
                cart.force_turn('^')
            elif char == 'v':
                cart.force_turn('<')
            elif char == '<':
                cart.force_turn('v')
        elif cell == '\\':
            if char == '^':
                cart.force_turn('<')
            elif char == '>':
                cart.force_turn('v')
            elif char == 'v':
                cart.force_turn('>')
            elif char == '<':
                cart.force_turn('^')
        elif cell == '+':
            cart.choose_turn()
        # Update cart location
        char = str(cart)
        new_row, new_col = row, col
        if char == '^':
            new_row -= 1
        elif char == 'v':
            new_row += 1
        elif char == '<':
            new_col -= 1
        elif char == '>':
            new_col += 1
        if (new_row, new_col) in carts:
            # CRASH!
            return new_row, new_col
        else:
            carts[new_row, new_col] = carts[row,col]
            del carts[row,col]

In [36]:
crash = move_carts(test_mat, test_carts)
if crash:
    print('CRASH', crash)
print_map(test_mat, test_carts)
for (row, col), test_cart in test_carts.items():
    print(row, col, repr(test_cart))

CRASH (3, 7)
/---\        
|   |  /----\
| /-+--+-\  |
| | |  v |  |
\-+-/  ^-+--/
  \------/   

4 7 <Cart direction=^ next_turn=0>
3 7 <Cart direction=v next_turn=2>


In [47]:
def find_crash(mat, carts):
    ''' Run the carts until they crash and display the location. '''
    while True:
        crash = move_carts(mat, carts)
        if crash:
            row, col = crash
            # My coordinates are reversed from the problem's
            print('Crash at {},{}'.format(col, row))
            break

In [48]:
test_text = r'''/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/'''
test_mat, test_carts = parse_map(test_text)
find_crash(test_mat, test_carts)

Crash at 7,3


In [49]:
with open('input.txt') as input_:
    mat, carts = parse_map(input_.read())
find_crash(mat, carts)

Crash at 116,10


# Part 2

In [67]:
def move_carts2(mat, carts):
    ''' Similiar to part 1 except we delete crashed carts instead of returning
    crash coordinates. '''
    # Make sure we process carts in the correct order.
    cart_list = [(row, col, cart) for (row,col), cart in carts.items()]
    cart_list.sort()
    to_deletes = list()
    for row, col, cart in cart_list:
        # Skip carts that are scheduled for deletion
        if (row, col) in to_deletes:
            continue
        cell = mat[row, col]
        char = str(cart)
        # Does the cart need to change direction?
        if cell == '/':
            if char == '^':
                cart.force_turn('>')
            elif char == '>':
                cart.force_turn('^')
            elif char == 'v':
                cart.force_turn('<')
            elif char == '<':
                cart.force_turn('v')
        elif cell == '\\':
            if char == '^':
                cart.force_turn('<')
            elif char == '>':
                cart.force_turn('v')
            elif char == 'v':
                cart.force_turn('>')
            elif char == '<':
                cart.force_turn('^')
        elif cell == '+':
            cart.choose_turn()
        # Update cart location
        char = str(cart)
        new_row, new_col = row, col
        if char == '^':
            new_row -= 1
        elif char == 'v':
            new_row += 1
        elif char == '<':
            new_col -= 1
        elif char == '>':
            new_col += 1
        if (new_row, new_col) in carts:
            # CRASH!
            # We don't want to modify dict while we're iterating it, so we
            # make a note to delete these carts later.
            to_deletes.append((row,col))
            to_deletes.append((new_row,new_col))
        else:
            carts[new_row, new_col] = carts[row,col]
            del carts[row,col]
    for to_delete in to_deletes:
        del carts[to_delete]

In [75]:
def find_last_cart(mat, carts):
    ''' Remove each pair of carts that crashes until only one cart remains. 
    Note this modifies the ``carts`` that you pass in! '''
    for n in itertools.count():
        move_carts2(mat, carts)
        if len(carts) == 1:
            break
        if n % 1000 == 0:
            print('n={} #carts={}'.format(n, len(carts)))
    for (row, col), cart in carts.items():
        print('Remaining cart {} at {},{}'.format(cart, col, row))

In [76]:
with open('input.txt') as input_:
    mat, carts = parse_map(input_.read())
find_last_cart(mat, carts)

n=0 #carts=17
n=1000 #carts=7
n=2000 #carts=5
n=3000 #carts=5
n=4000 #carts=5
n=5000 #carts=3
n=6000 #carts=3
n=7000 #carts=3
n=8000 #carts=3
n=9000 #carts=3
n=10000 #carts=3
Remaining cart ^ at 116,25
